In [ ]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
from snowflake import connector
from boto3 import client
from pandas import json_normalize
from dotenv import load_dotenv
import os
import requests
import boto3
import pandas as pd
from io import BytesIO
from pandas import json_normalize
from datetime import date

current_date = date.today()
print(current_date)


load_dotenv(override=True)  # This will reload and override existing variables
my_var = os.getenv("MY_VARIABLE")
print(my_var)
load_dotenv()
snowflake_user = os.getenv("SNOWFLAKE_USER")
snowflake_password = os.getenv("SNOWFLAKE_PASSWORD")
snowflake_account = os.getenv("SNOWFLAKE_ACCOUNT")
snowflake_warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
snowflake_database = os.getenv("SNOWFLAKE_DATABASE")
snowflake_schema_raw = os.getenv("SNOWFLAKE_SCHEMA_RAW")
snowflake_schema_stage = os.getenv("SNOWFLAKE_SCHEMA_STAGE")
snowflake_schema_raw_data= os.getenv("SNOWFLAKE_SCHEMA_RAW_DATA")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
bucket = os.getenv("BUCKET")
folder = os.getenv("FOLDER")
api_token = os.getenv("SERPSTAT_API_TOKEN")

client_name = 'ocf'

s3 = client("s3", aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)

def get_snowflake_connection(schema):
    return connector.connect(
        user=snowflake_user,
        password=snowflake_password,
        account=snowflake_account,
        warehouse=snowflake_warehouse,
        database=snowflake_database,
        schema=schema,
        role="ACCOUNTADMIN"
    )

def fetch_top_keywords(keywordCount):
    query = f"""
    WITH CTE AS (
    SELECT 
        QUERY, 
        COUNT(1) AS CNT, 
        SUM(IMPRESSIONS) AS IMPRESSIONS,
        ROW_NUMBER() OVER (PARTITION BY QUERY ORDER BY SUM(IMPRESSIONS) DESC) AS rank
    FROM GOOGLE_BIGQUERY.SEARCH_CONSOLE_RAW.QUERY
    GROUP BY QUERY
        )
        SELECT QUERY, CNT, IMPRESSIONS , rank
        FROM CTE
       --- WHERE IMPRESSIONS > 10
        ORDER BY IMPRESSIONS DESC 
        LIMIT {keywordCount}
        ;
    """
    conn = get_snowflake_connection(snowflake_schema_raw)
    try:
        cur = conn.cursor()
        cur.execute(query)
        results = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]
        df = pd.DataFrame(results, columns=column_names)
    finally:
        cur.close()
        conn.close()
    return df

def send_api_requests(keywordSearch ,  avoidedWords , numOfPages , sizOfPage , se , api_token):
    url = f"https://api.serpstat.com/v4/?token={api_token}"
    api_requests = [
        {
            "description": "getKeywords",
            "method": "SerpstatKeywordProcedure.getKeywords",
                      "params": {
                                    "keyword": keywordSearch,  # Replace with actual keyword
                                    "se": se,
                                    "minusKeywords": avoidedWords,
                                    "filters": {
                                        "cost_from": 1,
                                        "cost_to": 10,
                                        "region_queries_count_from": 300,
                                        "region_queries_count_to": 2000
                                    },
                                    "page": numOfPages,
                                    "size": sizOfPage}
                                    ,
            "result_path": "result['result']['data']",
            "snowflake_table": "SERPSTAT.KEYWORD_RAW_DATA.GETKEYWORDS",
            "stage_table_shortname": "GETKEYWORDS_STG",
        },
        {
            "description": "getSuggestions",
            "method": "SerpstatKeywordProcedure.getSuggestions",
            "params": {
                        "keyword": keywordSearch,  # Replace with actual keyword
                        "se": se,
                        "page": numOfPages,
                        "size": sizOfPage,
                                    },
            "result_path": "result['result']['data']",
            "snowflake_table": "SERPSTAT.KEYWORD_RAW_DATA.GETSUGGESTIONS",
            "stage_table_shortname": "GETSUGGESTIONS_STG",
        },
        {
            "description": "getRelatedKeywords",
            "method": "SerpstatKeywordProcedure.getRelatedKeywords",
            "params": {
                    "keyword": keywordSearch,  # Replace with actual keyword
                    "se": se,
                    "filters": {
                        "cost_from": 0,
                        "cost_to": 5,
                        "difficulty_from": 5,
                        "difficulty_to": 30
                    },
                    "page": numOfPages,
                    "size": sizOfPage
                                },
            "result_path": "result['result']['data']",
            "snowflake_table": "SERPSTAT.KEYWORD_RAW_DATA.GETRELATEDKEYWORDS",
            "stage_table_shortname": "GETRELATEDKEYWORDS_STG",
        }
    ]
    return api_requests

# Upload DataFrame to S3 and return path
def upload_to_s3(df, description, index):
    file_name = f"{description}/csv_{index}.csv"
    df.to_csv(file_name, index=False)
    s3.upload_file(file_name, bucket, f"{folder}/{file_name}")
    os.remove(file_name)
    return f"{folder}/{file_name}"

# Process to Snowflake and track
def process_to_snowflake(conn, s3_path, description):
    copy_command = f"""
    COPY INTO {description}
    FROM 's3://{bucket}/{s3_path}'
    CREDENTIALS = (AWS_KEY_ID='{aws_access_key_id}' AWS_SECRET_KEY='{aws_secret_access_key}')
    FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER=1)
    ON_ERROR='CONTINUE';
    """
    with conn.cursor() as cur:
        cur.execute(copy_command)
        print(f"Data successfully copied to {description}")



def upload_df_to_s3(df, description):
    csv_buffer = BytesIO()
    df.to_csv(csv_buffer, index=False)
    csv_buffer.seek(0)
    file_name = f"{description}/data.csv"
    s3_path = f"{folder}/{file_name}"
    s3.put_object(Bucket=bucket, Key=s3_path, Body=csv_buffer.getvalue())
    
    print(f"Data uploaded to s3://{bucket}/{s3_path}")




url = f"https://api.serpstat.com/v4/?token={api_token}"

top_keywords = fetch_top_keywords(1000)

size = 100 
avoidedWords = ['kellogs', 'corn flakes']
numOfPages = 5
sizOfPage = 20
se = 'g_us'

relatedKeyWordList = []
getSuggestionsList = []
getKeywordsList = []

# Loop through keywords and send API requests
for keyword in top_keywords['QUERY'][:1000]:
    for index, api_call in enumerate(send_api_requests(keyword,  avoidedWords, numOfPages, sizOfPage, se, api_token), 1):
        result_path = api_call['result_path']  # Extract result_path
        description = api_call['description']
        method = api_call['method']
        params = api_call['params']

        try:
            response = requests.post(
                url,
                headers={"Authorization": f"Bearer {api_token}"},
                json={"id": 1, "method": method, "params": params}
            )
            response.raise_for_status()  # Check if the request was successful
            if description == 'getRelatedKeywords':
                try:
                    data = response.json()['result']['data']
                    data = json_normalize(data)
                    data['keyword_searched'] = keyword
                    relatedKeyWordList.append(data)
                except (KeyError, ValueError) as e:
                    print(f"Error processing 'getRelatedKeywords' for {keyword}: {e}")

            elif description == 'getSuggestions':
                try:
                    data = response.json()['result']['data']
                    data = json_normalize(data)
                    data['keyword_searched'] = keyword
                    getSuggestionsList.append(data)
                except (KeyError, ValueError) as e:
                    print(f"Error processing 'getSuggestions' for {keyword}: {e}")

            elif description == 'getKeywords':
                try:
                    data = response.json()['result']['data']
                    data = json_normalize(data)
                    data['keyword_searched'] = keyword
                    getKeywordsList.append(data)
                except (KeyError, ValueError) as e:
                    print(f"Error processing 'getKeywords' for {keyword}: {e}")

        except requests.exceptions.RequestException as e:
            print(f"Request failed for {keyword} with method {method}: {e}")
        except (KeyError, ValueError) as e:
            print(f"Unexpected response format for {keyword} with method {method}: {e}")

relatedKeyWorddf = pd.concat(relatedKeyWordList)
getSuggestionsListdf = pd.concat(getSuggestionsList)
getKeywordsListdf = pd.concat(getKeywordsList)

getSuggestionsListdf = getSuggestionsListdf.drop(['geo_names'], axis=1)
relatedKeyWorddf = relatedKeyWorddf.drop(['geo_names','types'], axis=1)
getKeywordsListdf = getKeywordsListdf.drop(['geo_names','types','social_domains'], axis=1)

getKeywordsListdf = getKeywordsListdf.drop_duplicates()
relatedKeyWorddf = relatedKeyWorddf.drop_duplicates()
getSuggestionsListdf = getSuggestionsListdf.drop_duplicates()



relatedKeyWorddf = pd.concat(relatedKeyWordList).drop(['geo_names', 'types'], axis=1).drop_duplicates()
getSuggestionsListdf = pd.concat(getSuggestionsList).drop(['geo_names'], axis=1).drop_duplicates()
getKeywordsListdf = pd.concat(getKeywordsList).drop(['geo_names', 'types', 'social_domains'], axis=1).drop_duplicates()

relatedKeyWorddf['client'] = client_name 
getSuggestionsListdf['client'] = client_name 
getKeywordsListdf['client'] = client_name 

relatedKeyWorddf['load_timestamp'] = ''
getSuggestionsListdf['load_timestamp'] = ''
getKeywordsListdf['load_timestamp'] = ''

relatedKeyWorddf['timestamp'] = current_date
getSuggestionsListdf['timestamp'] = current_date
getKeywordsListdf['timestamp'] = current_date


dataframes = [
    (relatedKeyWorddf, 'related_keywords'),
    (getSuggestionsListdf, 'suggestions'),
    (getKeywordsListdf, 'keywords')
]
for df, description in dataframes:
    upload_df_to_s3(df, description)

2024-11-12
None


# S3 to Snowflake Staging

In [ ]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
from snowflake import connector
from boto3 import client
from pandas import json_normalize
from dotenv import load_dotenv
import os
import requests
import boto3
import pandas as pd
from io import BytesIO
from pandas import json_normalize

client = 'ocf'
load_dotenv(override=True)  # This will reload and override existing variables
my_var = os.getenv("MY_VARIABLE")
print(my_var)
load_dotenv()
snowflake_user = os.getenv("SNOWFLAKE_USER")
snowflake_password = os.getenv("SNOWFLAKE_PASSWORD")
snowflake_account = os.getenv("SNOWFLAKE_ACCOUNT")
snowflake_warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
snowflake_database = os.getenv("SNOWFLAKE_DATABASE")
snowflake_schema_raw = os.getenv("SNOWFLAKE_SCHEMA_RAW")
snowflake_schema_stage = os.getenv("SNOWFLAKE_SCHEMA_STAGE")
snowflake_schema_raw_data= os.getenv("SNOWFLAKE_SCHEMA_RAW_DATA")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
bucket = os.getenv("BUCKET")
folder = os.getenv("FOLDER")
api_token = os.getenv("SERPSTAT_API_TOKEN")



def get_snowflake_connection(schema):
    return connector.connect(
        user=snowflake_user,
        password=snowflake_password,
        account=snowflake_account,
        warehouse=snowflake_warehouse,
        database=snowflake_database,
        schema=schema,
        role="ACCOUNTADMIN"
    )

def snowflakeInformation():
    api_requests = [
        {
            "blob_path": "s3://serp-api-blob/Query/keywords/data.csv",
            "snowflake_table": "SERPSTAT.KEYWORD_RAW_DATA.GETKEYWORDS",
            "stage_table_shortname": "GETKEYWORDS_STG",
        },
        {
            "blob_path": "s3://serp-api-blob/Query/suggestions/data.csv",
            "snowflake_table": "SERPSTAT.KEYWORD_RAW_DATA.GETSUGGESTIONS",
            "stage_table_shortname": "GETSUGGESTIONS_STG",
        },
        {
            "blob_path": "s3://serp-api-blob/Query/related_keywords/data.csv",
            "snowflake_table": "SERPSTAT.KEYWORD_RAW_DATA.GETRELATEDKEYWORDS",
            "stage_table_shortname": "GETRELATEDKEYWORDS_STG",
        }
    ]
    return api_requests

def execute_snowflake_copy(conn):
    for api_call in snowflakeInformation():
        blob_path = api_call['blob_path']
        stage_table = api_call['stage_table_shortname']
        
        # Build the COPY command
        copy_command = f"""
        COPY INTO {stage_table}
        FROM '{blob_path}'
        CREDENTIALS = (AWS_KEY_ID='{aws_access_key_id}' AWS_SECRET_KEY='{aws_secret_access_key}')
        FILE_FORMAT = (TYPE = 'CSV' FIELD_OPTIONALLY_ENCLOSED_BY = '"' SKIP_HEADER=1)
        ON_ERROR='CONTINUE';
        """
        try:
            with conn.cursor() as cur:
                cur.execute(copy_command)
                print(f"Data successfully copied to {stage_table}")
        except Exception as e:
            print(f"Failed to copy data to {stage_table}: {e}")

def timestamping(conn):
    for api_call in snowflakeInformation():
        stage_table = api_call['stage_table_shortname']
        update_command = f"""
        UPDATE {stage_table}
        SET load_timestamp = CURRENT_TIMESTAMP
        WHERE load_timestamp IS NULL;
        """
        try:
            with conn.cursor() as cur:
                cur.execute(update_command)
                print(f"Timestamp updated in {stage_table}")
        except Exception as e:
            print(f"Error updating timestamp in {stage_table}: {e}")

def merge_queries():
    merges = ["""
    MERGE INTO SERPSTAT.KEYWORD_RAW_DATA.GETRELATEDKEYWORDS AS target
    USING SERPSTAT.KEYWORD_STAGE_TABLES.GETRELATEDKEYWORDS_STG AS source
    ON target.keyword = source.keyword AND target.timestamp =  source.timestamp
    WHEN MATCHED THEN
        UPDATE SET
            target.region_queries_count = source.region_queries_count,
            target.cost = source.cost,
            target.concurrency = source.concurrency,
            target.right_spelling = source.right_spelling,
            target.weight = source.weight,
            target.difficulty = source.difficulty,
            target.client = source.client,
            target.load_timestamp = source.load_timestamp,
            target.timestamp = source.timestamp,
            target.keyword_searched = source.keyword_searched
    -- Insert new records
    WHEN NOT MATCHED THEN
        INSERT (
            keyword,
            region_queries_count,
            cost,
            concurrency,
            right_spelling,
            weight,
            difficulty,
            keyword_searched ,
            client,
            load_timestamp,
            timestamp
        )
        VALUES (
            source.keyword,source.region_queries_count, source.cost,source.concurrency,
            source.right_spelling, source.weight,source.difficulty,source.keyword_searched , source.client, source.load_timestamp,source.timestamp
        );
    """ , 

    """
    MERGE INTO SERPSTAT.KEYWORD_RAW_DATA.GETKEYWORDS target
    USING SERPSTAT.KEYWORD_STAGE_TABLES.GETKEYWORDS_STG source
    ON target.keyword = source.keyword AND target.timestamp =  source.timestamp
    WHEN MATCHED THEN UPDATE SET target.cost = source.cost, target.concurrency = source.concurrency, 
    target.found_results = source.found_results, target.region_queries_count = source.region_queries_count, 
    target.region_queries_count_wide = source.region_queries_count_wide, target.right_spelling = source.right_spelling, 
    target.lang = source.lang, target.keyword_length = source.keyword_length, target.difficulty = source.difficulty, 
    target.client = source.client, target.load_timestamp = source.load_timestamp, target.timestamp = source.timestamp, target.keyword_searched = source.keyword_searched
    WHEN NOT MATCHED THEN INSERT (keyword, cost, concurrency, found_results, region_queries_count, region_queries_count_wide, right_spelling, lang, keyword_length, difficulty,keyword_searched ,  client, load_timestamp, timestamp) 
    VALUES (source.keyword, source.cost, source.concurrency, source.found_results, source.region_queries_count, source.region_queries_count_wide, source.right_spelling, source.lang, source.keyword_length, source.difficulty,source.keyword_searched, source.client, source.load_timestamp, source.timestamp);
    """ ,
    """
            MERGE INTO SERPSTAT.KEYWORD_RAW_DATA.GETSUGGESTIONS AS target
        USING SERPSTAT.KEYWORD_STAGE_TABLES.GETSUGGESTIONS_STG AS source
        ON target.keyword = source.keyword 
        AND target.keyword_searched = source.keyword_searched 
        AND target.timestamp = source.timestamp AND t
        arget.timestamp =  source.timestamp
        WHEN MATCHED THEN 
            UPDATE SET 
                target.client = source.client, 
                target.timestamp = source.timestamp, 
                target.load_timestamp = source.load_timestamp
        WHEN NOT MATCHED THEN 
            INSERT (keyword, keyword_searched, client, timestamp, load_timestamp) 
            VALUES (source.keyword, source.keyword_searched, source.client, source.timestamp, source.load_timestamp);
    """
    ]
    return merges

def delete_file_from_s3(path):
    try:
        s3.delete_object(Bucket=bucket, Key=path)
        print(f"File deleted from s3://{path}")
    except Exception as e:
        print(f"Error deleting file: {e}")


def merging_data(conn):
    for merge in merge_queries():
        try:
            with conn.cursor() as cur:
                cur.execute(merge)
                cur.execute(merge)

        except Exception as e:
            print(f"Error updating timestamp")
    with conn.cursor() as cur:
        cur.execute("truncate SERPSTAT.KEYWORD_STAGE_TABLES.GETSUGGESTIONS_STG ")
        cur.execute("truncate SERPSTAT.KEYWORD_STAGE_TABLES.GETKEYWORDS_STG ")
        cur.execute("truncate SERPSTAT.KEYWORD_STAGE_TABLES.GETRELATEDKEYWORDS_STG ")

execute_snowflake_copy(get_snowflake_connection(snowflake_schema_stage))
timestamping(get_snowflake_connection(snowflake_schema_stage))
merging_data(get_snowflake_connection(snowflake_schema_stage))

for api_call in snowflakeInformation():
    blob_path = api_call['blob_path']
    stage_table = api_call['stage_table_shortname']
    
    delete_file_from_s3(blob_path)

None


# Blog Model

In [ ]:
with open('requirements.txt', 'r') as file:
    packages = file.read().splitlines()

# List of standard libraries that shouldn't be installed with pip
standard_libs = {"json", "os", "sys", "re", "time"}  # add more if necessary
for package in packages:
    # Skip standard library modules
    if package in standard_libs:
        print(f"Skipping standard library module: {package}")
        continue
    try:
        subprocess.check_call(['pip', 'install', package])
        print(f"Successfully installed {package}")
    except subprocess.CalledProcessError as e:
        print(f"Failed to install {package}. Error: {e}")



In [ ]:
import os
import time
import requests
import pandas as pd
from dotenv import load_dotenv
from datetime import datetime
from snowflake import connector
from boto3 import client
from pandas import json_normalize
from dotenv import load_dotenv
import os
import requests
import boto3
import pandas as pd
from io import BytesIO
from pandas import json_normalize
import subprocess
import openai 

client = 'ocf'
load_dotenv(override=True)  # This will reload and override existing variables
my_var = os.getenv("MY_VARIABLE")
print(my_var)
load_dotenv()
snowflake_user = os.getenv("SNOWFLAKE_USER")
snowflake_password = os.getenv("SNOWFLAKE_PASSWORD")
snowflake_account = os.getenv("SNOWFLAKE_ACCOUNT")
snowflake_warehouse = os.getenv("SNOWFLAKE_WAREHOUSE")
snowflake_database = os.getenv("SNOWFLAKE_DATABASE")
snowflake_schema_raw = os.getenv("SNOWFLAKE_SCHEMA_RAW")
snowflake_schema_stage = os.getenv("SNOWFLAKE_SCHEMA_STAGE")
snowflake_schema_raw_data= os.getenv("SNOWFLAKE_SCHEMA_RAW_DATA")
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
bucket = os.getenv("BUCKET")
folder = os.getenv("FOLDER")
api_token = os.getenv("SERPSTAT_API_TOKEN")
#openAiKey = 'sk-'
#os.getenv("SERPSTAT_API_TOKEN")
#openai.api_key = 'sk-'


def get_snowflake_connection(schema):
    return connector.connect(
        user=snowflake_user,
        password=snowflake_password,
        account=snowflake_account,
        warehouse=snowflake_warehouse,
        database=snowflake_database,
        schema=schema,
        role="ACCOUNTADMIN"
    )

current_date = datetime.now().strftime('%Y-%m-%d')

url = f"https://api.serpstat.com/v4/?token={api_token}"


staging_table = "CONTENT.STAGING_TABLES.PRODUCT_CONTENTS"
batch_size = 100000
snowflake_table = "CONTENT.BLOG_POSTS.PRODUCT_CONTENTS"
company_id= 1

def query(conn,limit):
    cur = conn.cursor()
    query = f""" 
       select QUERY, 
            KW.KEYWORD AS KW_KEYWORD ,
            R.KEYWORD AS R_KEYWORD, 
            GS.KEYWORD_SEARCHED ,
            SUM(IMPRESSIONS) AS IMPRESSIONS, 
            SUM(CLICKS) AS CLICKS 
            from GOOGLE_BIGQUERY.SEARCH_CONSOLE_RAW.QUERY Q 
            LEFT JOIN  SERPSTAT.KEYWORD_RAW_DATA.GETKEYWORDS KW ON Q.QUERY = KW.KEYWORD_SEARCHED
            LEFT JOIN SERPSTAT.KEYWORD_RAW_DATA.GETRELATEDKEYWORDS R ON Q.QUERY =  R.KEYWORD_SEARCHED
            LEFT JOIN  SERPSTAT.KEYWORD_RAW_DATA.GETSUGGESTIONS GS ON Q.QUERY = GS.KEYWORD_SEARCHED
            WHERE KW.KEYWORD IS NOT NULL AND R.KEYWORD  IS NOT NULL
            GROUP BY 1,2,3,4
            ORDER BY 5 DESC 
            LIMIT {limit};
            """  
    cur.execute(query)
    results = cur.fetchall()
    column_names = [desc[0] for desc in cur.description]
    df = pd.DataFrame(results, columns=column_names)
    return df 

def generate_text(prompt, max_tokens=150):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",  # You can use "gpt-4" if available on your account
        messages=[
            {"role": "system", "content": "You are a helpful assistant."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        n=1,
        stop=None,
        temperature=0.7  # Controls randomness (higher = more random)
    )
    return response['choices'][0]['message']['content'].strip()

def generate_blog_post(product_name, product_description, suggested_keywords, query_keywords):
    prompt = (f"Write a detailed blog post about {product_name}. "
              f"Description: {product_description}. Highlight the following: {suggested_keywords}, "
              f"and target the following search queries: {query_keywords}. "
              "Mention that readers can also check out our social media post for more updates.")
    return generate_text(prompt, max_tokens=400)


def generate_social_post(product_name, product_description, suggested_keywords, query_keywords):
    hashtags = f"#{suggested_keywords.replace(' ', '')} #{query_keywords.replace(' ', '')}"
    
    prompt = (f"Write a short social media post about {product_name}. "
              f"Description: {product_description}. Focus on {suggested_keywords} and use the keywords "
              f"{query_keywords}. Include a call to action telling users to check out our blog post for more details. "
              "Also, generate a caption and relevant hashtags.")
    
    social_post = generate_text(prompt, max_tokens=100)
    
    # Append hashtags and call to action for blog post
    full_social_post = f"{social_post}\n\nRead more in our blog post! [Insert Blog Link Here]\n{hashtags}"
    return full_social_post

final_df = pd.DataFrame()

counter = 0

df = query(get_snowflake_connection(snowflake_schema_raw_data), 10)


None


In [ ]:

# Define your folder ID
FOLDER_ID = '1DgNAor9qfPubo808ia_N8gW-oOhmPcUb'

# Authenticate with Google Drive API
SCOPES = ['https://www.googleapis.com/auth/drive']
creds = service_account.Credentials.from_service_account_file('surremor.json', scopes=SCOPES)
service = build('drive', 'v3', credentials=creds)

# List all files in the folder
def list_files_in_folder(folder_id):
    query = f"'{folder_id}' in parents"
    results = service.files().list(
        q=query,
        spaces='drive',
        fields='files(id, name, mimeType)',
    ).execute()
    return results.get('files', [])

# Download and export files as needed
def download_or_export_file(file_id, file_name, mime_type):
    # Handle Google Docs Editors files by exporting them in the appropriate format
    if mime_type == 'application/vnd.google-apps.spreadsheet':
        request = service.files().export(fileId=file_id, mimeType='application/vnd.openxmlformats-officedocument.spreadsheetml.sheet')
        file_name = f"{file_name}.xlsx"
    elif mime_type == 'application/vnd.google-apps.document':
        request = service.files().export(fileId=file_id, mimeType='application/pdf')
        file_name = f"{file_name}.pdf"
    else:
        request = service.files().get_media(fileId=file_id)

    # Download the file
    fh = io.FileIO(file_name, 'wb')
    downloader = MediaIoBaseDownload(fh, request)
    
    done = False
    while not done:
        status, done = downloader.next_chunk()
        print(f"Downloading {file_name}: {int(status.progress() * 100)}% complete.")
    
    fh.close()
    print(f"Downloaded {file_name}")
    return file_name

# Download all files in the folder, export Google Sheets to DataFrame
def download_all_files_from_folder(folder_id):
    files = list_files_in_folder(folder_id)
    dataframes = []
    
    for file in files:
        file_id = file['id']
        file_name = file['name']
        mime_type = file['mimeType']
        
        # Download or export the file based on type
        downloaded_file = download_or_export_file(file_id, file_name, mime_type)
        
        # Load exported Google Sheets files and CSV files into DataFrames
        if downloaded_file.endswith('.xlsx'):
            df = pd.read_excel(downloaded_file)
            dataframes.append(df)
        elif downloaded_file.endswith('.csv'):
            df = pd.read_csv(downloaded_file)
            dataframes.append(df)
        
        # Optionally, delete the downloaded file after loading
        os.remove(downloaded_file)
    
    # Combine all DataFrames into one if applicable
    combined_df = pd.concat(dataframes, ignore_index=True) if dataframes else None
    return combined_df

# Run the download and export function
combined_df = download_all_files_from_folder(FOLDER_ID)

# Display the combined DataFrame if data was loaded
if combined_df is not None:
    print(combined_df.head())
else:
    print("No compatible files (CSV/Excel) found in the folder.")


No compatible files (CSV/Excel) found in the folder.


In [121]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sample data (replace this with your actual loaded data)
# combined_df and df are assumed to be already loaded

# Step 1: Calculate the cosine similarity
def get_best_match(combined_df, df):
    # Vectorize the text columns
    vectorizer = TfidfVectorizer()
    combined_descriptions = combined_df['description'].values
    df_queries = df['QUERY'].values
    
    # Fit and transform the text data
    combined_vec = vectorizer.fit_transform(combined_descriptions)
    df_vec = vectorizer.transform(df_queries)
    
    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(combined_vec, df_vec)
    
    # Find the index of the most similar row in df for each row in combined_df
    best_matches = similarity_matrix.argmax(axis=1)
    best_match_scores = similarity_matrix.max(axis=1)
    
    # Get the most similar row in df for each row in combined_df
    best_match_df = df.iloc[best_matches].reset_index(drop=True)
    best_match_df['similarity_score'] = best_match_scores  # Add similarity score for reference
    
    # Concatenate with combined_df for a left join effect
    merged_df = pd.concat([combined_df.reset_index(drop=True), best_match_df], axis=1)
    
    return merged_df

# Perform the left join with cosine similarity matching
merged_df = get_best_match(combined_df, df)

# Display the result
print(merged_df)


     category    product description  variant       QUERY   
0  fertilizer  greensand  green sand        5  green sand  \

                    KW_KEYWORD           R_KEYWORD KEYWORD_SEARCHED   
0  green sand water filtration  what is green sand       green sand  \

   IMPRESSIONS  CLICKS  similarity_score  
0           39       0               1.0  


In [111]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import openai

# Function to check for uniqueness using cosine similarity
def is_unique(new_content, previous_posts, similarity_threshold=0.7):
    if not previous_posts:
        return True  # If there are no previous posts, the content is unique by default

    # Vectorize the previous posts and the new content together
    vectorizer = TfidfVectorizer().fit_transform(previous_posts + [new_content])
    cosine_matrix = cosine_similarity(vectorizer[-1], vectorizer[:-1])
    
    # Find the maximum similarity score
    max_similarity = cosine_matrix.max()
    
    # If max similarity is below the threshold, content is unique
    return max_similarity < similarity_threshold

# Function to generate unique blog content
def generate_unique_blog_content(query, kw_keyword, r_keyword, keyword_searched, previous_posts):
    attempt = 0
    max_attempts = 3
    unique_content = None

    while attempt < max_attempts:
        prompt = (f"Write a unique blog post about '{query}', using the following keywords naturally in the content:\n\n"
                  f"KW_KEYWORD: {kw_keyword}\n"
                  f"R_KEYWORD: {r_keyword}\n"
                  f"KEYWORD_SEARCHED: {keyword_searched}\n\n"
                  "Create a unique perspective with new examples, tips, or user stories that differ from similar topics.")
        
        response = openai.ChatCompletion.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.7
        )
        generated_content = response.choices[0].message['content']

        # Check uniqueness against previous posts using cosine similarity
        if is_unique(generated_content, previous_posts):
            unique_content = generated_content
            break

        attempt += 1
    return unique_content

# List to store each post's data for the DataFrame
output_data = []
previous_posts = []  # Stores content for similarity checking

# Generate content for each row in the DataFrame
for _, row in df.iterrows():
    blog_post = generate_unique_blog_content(
        row["QUERY"],
        row["KW_KEYWORD"],
        row["R_KEYWORD"],
        row["KEYWORD_SEARCHED"],
        previous_posts
    )
    output_data.append({
        "Query": row["QUERY"],
        "KW_KEYWORD": row["KW_KEYWORD"],
        "R_KEYWORD": row["R_KEYWORD"],
        "KEYWORD_SEARCHED": row["KEYWORD_SEARCHED"],
        "Blog Post Content": blog_post
    })
    if blog_post:  # Add only if content was generated
        previous_posts.append(blog_post)  # Add generated post to list for future similarity checks
# Create DataFrame from output data
blog_posts_df = pd.DataFrame(output_data)

# Display the DataFrame
print(blog_posts_df)

         Query                        KW_KEYWORD           R_KEYWORD   
0   green sand       green sand water filtration  what is green sand  \
1  corn gluten  gluten free corn flakes kelloggs   weeds corn gluten   

  KEYWORD_SEARCHED                                  Blog Post Content  
0       green sand  Title: The Magic of Green Sand: A Natural Powe...  
1      corn gluten  Title: The Power of Corn Gluten: From Kellogg'...  


# DRIVE UPLOAD

In [128]:
import pandas as pd
from google.oauth2 import service_account
from googleapiclient.discovery import build
from googleapiclient.http import MediaFileUpload
from datetime import datetime

# Get the current date and format it
current_date = datetime.now().strftime("%Y%m%d")

FOLDER_ID = '1C8Aym9Nh9rxeFdje6-7IK98pdJZtPAWL'
csv_path = f'blog_posts_{current_date}.csv'

# Save the DataFrame as CSV
blog_posts_df.to_csv(csv_path, index=False)

# Authenticate and create the Google Drive API client
SCOPES = ['https://www.googleapis.com/auth/drive.file']
creds = service_account.Credentials.from_service_account_file('surremor.json', scopes=SCOPES)
service = build('drive', 'v3', credentials=creds)

# Function to upload CSV to Google Drive
def upload_csv_to_drive(file_path, folder_id):
    file_metadata = {
        'name': file_path.split('/')[-1],  # Name in Drive
        'parents': [folder_id]  # Target folder ID in Drive
    }
    
    # Set the MIME type to 'text/csv' to ensure it stays as a CSV
    media = MediaFileUpload(file_path, mimetype='text/csv', resumable=True)
    
    uploaded_file = service.files().create(
        body=file_metadata,
        media_body=media,
        fields='id'
    ).execute()
    print(f"File uploaded successfully with ID: {uploaded_file.get('id')}")

# Run the upload function
upload_csv_to_drive(csv_path, FOLDER_ID)




File uploaded successfully with ID: 1BY8e7iW0_tjJZPd80VGQNLT4aa8fwXse
